pretrain affine Q network

In [ ]:
using Revise
using Pkg
Pkg.activate("/home/jiaxingl/project/verify_julia_env")
Pkg.status()

  Activating project at `~/project/verify_julia_env`


Status `~/project/verify_julia_env/Project.toml`
  [d8c2afa5] Cersyve v1.0.0-DEV `~/project/Cersyve.jl`
⌅ [587475ba] Flux v0.13.17
  [f67ccb44] HDF5 v0.17.2
  [7073ff75] IJulia v1.26.0
⌅ [033835bb] JLD2 v0.4.53
  [6d061d49] ModelVerification v0.1.0 `/home/jiaxingl/project/ModelVerification.jl#cersyve`
  [85610aed] NaiveNASflux v2.0.8 `~/project/ModelVerification.jl/onnx_parser/NaiveNASflux`
⌃ [bd45eb3e] NaiveNASlib v2.0.11
⌃ [d0dd6a25] ONNX v0.2.0
  [2e935253] ONNXNaiveNASflux v0.2.7 `~/project/ModelVerification.jl/onnx_parser/ONNXNaiveNASflux`
  [438e738f] PyCall v1.96.4
  [9a3f8284] Random
Info Packages marked with ⌃ and ⌅ have new versions available, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [ ]:
using Cersyve
using Flux
using JLD2
using Random

# struct FilterX
#     W::Matrix  # Weight matrix
# end

# struct FilterU
#     W::Matrix  # Weight matrix
# end

# function (layer::FilterX)(input::Matrix{Float32})
#     return layer.W * input
# end

# Flux.@functor FilterX  # Make the layer compatible with Flux
# function Flux.params(layer::FilterX)
#     return Flux.Params([])  # Exclude weights from being trainable
# end

# # Define a filtering layer for extracting u (indices 9 to 14)


# function (layer::FilterU)(input::Matrix{Float32})
#     return layer.W * input
# end

# Flux.@functor FilterU  # Make the layer compatible with Flux
# function Flux.params(layer::FilterU)
#     return Flux.Params([])  # Exclude weights from being trainable
# end

# Initialize the fixed weight matrices for filtering
function create_filter_matrix(start_idx, end_idx, total_len)
    W = zeros(end_idx - start_idx + 1, total_len)
    for i in start_idx:end_idx
        W[i - start_idx + 1, i] = 1.0
    end
    return W
end

function create_parallel_affine_Q(x_dim, u_dim)
    # Assume the input has 13 elements: x (0–7), u (8–13)
    x_w = create_filter_matrix(1, x_dim, x_dim+u_dim)
    x_b = zeros(x_dim)
    filter_x = Dense(x_w, x_b)

    u_w = create_filter_matrix(x_dim, x_dim+u_dim-1, x_dim+u_dim)
    u_b = zeros(u_dim)
    filter_u = Dense(u_w, u_b)
    
    #Branch1
    b1 = Chain(
        filter_x,  # First hidden layer (32 neurons, input size is 8 for x)
        Dense(x_dim, 32, relu),  # First hidden layer (32 neurons, input size is 8 for x)
        Dense(32, 32, relu)  # Second hidden layer (32 neurons)
    )

    # Define the final output layer (scalar output)
    final_layer = Chain(Dense(32 + u_dim, 1))  # Concatenation of x (32) and u (6)

    # Complete model
    model = Chain(
        Parallel(
            vcat, 
            b1,
            filter_u
        ),
        final_layer                       # Compute scalar output
    )
    return model
end





task = Unicycle
value_hidden_sizes = [32, 32]
dynamics_hidden_sizes = [32, 32]
constraint_hidden_sizes = [16]
data_path = joinpath(@__DIR__, "../data/unicycle_data.jld2")
model_dir = joinpath(@__DIR__, "../model/unicycle/")
log_dir = joinpath(@__DIR__, "../log/unicycle/")
seed = 1

Random.seed!(seed)

# V_model = Cersyve.create_mlp(task.x_dim, 1, value_hidden_sizes)
# Q_model = Cersyve.create_mlp(task.x_dim + task.u_dim, 1, value_hidden_sizes)


data = JLD2.load(data_path)["data"]
f_model = Cersyve.create_mlp(task.x_dim + task.u_dim, task.x_dim, dynamics_hidden_sizes)
Flux.loadmodel!(f_model, JLD2.load(joinpath(model_dir, "f.jld2"), "state"))
f_pi_model = Cersyve.create_closed_loop_dynamics_model(
    f_model, task.pi_model, data, task.x_low, task.x_high, task.u_dim)

h_model = Cersyve.create_mlp(task.x_dim, 1, constraint_hidden_sizes)
Flux.loadmodel!(h_model, JLD2.load(joinpath(model_dir, "h.jld2"), "state"))

x_a_low =  [task.x_low; task.u_low]
x_a_high = [task.x_high; task.u_high]


affine_Q = create_parallel_affine_Q(task.x_dim, task.u_dim)

# trainable parameters
# println(affine_Q[1][1][2])
# println(affine_Q[1][1][3])
# println(affine_Q[2])

# pretrain_Q(
#     affine_Q,
#     f_pi_model,
#     task.pi_model,
#     h_model,
#     task.x_low,
#     task.x_high;
#     penalty="APA",
#     space_size=x_a_high - x_a_low,
#     apa_coef=1e-4,
#     log_dir=log_dir,
# )




Chain(
  Parallel(
    vcat,
    Chain(
      Dense(5 => 3),                    # 18 parameters
      Dense(3 => 32, relu),             # 128 parameters
      Dense(32 => 32, relu),            # 1_056 parameters
    ),
    Dense(5 => 2),                      # 12 parameters
  ),
  Chain(
    Dense(34 => 1),                     # 35 parameters
  ),
)                   # Total: 10 arrays, 1_249 parameters, 5.871 KiB.

Testing Q_Q_next model where Q_next is calculating argmin_u (Q) with interval arithmatics

In [23]:
struct ExpandXToXU
    x_dim::Int
    u_dim::Int
end

# Define the forward pass for the layer
function (layer::ExpandXToXU)(x)
    # `x` is the input vector of size `x_dim`
    u = rand(layer.u_dim)  # Generate random `u` part
    return vcat(x, u)      # Concatenate `x` and `u`
end

# Example usage
function create_expand_xu_layer(x_dim, u_dim)
    return ExpandXToXU(x_dim, u_dim)
end

# Custom Dense Layer for Interval Arithmetic
struct DenseInterval
    W::AbstractMatrix
    b::AbstractVector
    x_dim::Int
    u_low::AbstractVector
    u_up::AbstractVector
end

function DenseInterval(W, b, x_dim, u_low, u_up)
    DenseInterval(W, b, x_dim, u_low, u_up)
end

# Forward pass for interval arithmetic
function (layer::DenseInterval)(xu::AbstractVector)
    # Compute lower and upper bounds for each neuron
    x_dim = size(xu)[1] - size(layer.u_low)[1]
    println(x_dim)
    x = xu[1:x_dim]
    println(size(x))
    W_u = layer.W[:, x_dim+1:end]
    W_up = max.(W_u, 0.0)
    W_un = min.(W_u, 0.0)
    
    W_p = layer.W
    W_p[:, x_dim+1:end] .= W_up
    W_n = layer.W
    W_n[:, x_dim+1:end] .= W_un

    xu_low = vcat(x, layer.u_low)
    xu_high = vcat(x, layer.u_up)
    l = W_p * xu_low .+ W_n * xu_high .+ layer.b
    return l
end

# Create the model
function create_parallel_affine_Q_interval(x_dim, u_dim, u_low, u_up)
    # Assume the input has 13 elements: x (0–7), u (8–13)
    x_w = create_filter_matrix(1, x_dim, x_dim+u_dim)
    x_b = zeros(x_dim)
    filter_x = Dense(x_w, x_b)

    u_w = create_filter_matrix(x_dim, x_dim+u_dim-1, x_dim+u_dim)
    u_b = zeros(u_dim)
    filter_u = Dense(u_w, u_b)
    
    # Branch 1
    b1 = Chain(
        filter_x,
        Dense(x_dim, 32, relu),
        Dense(32, 32, relu)
    )

    # Define the interval arithmetic layer for the final output
    final_layer_interval = DenseInterval(randn(1, 32 + u_dim), randn(1), task.x_dim, task.u_low, task.u_high)

    # Complete model
    model = Chain(
        Parallel(
            vcat, 
            b1,
            filter_u
        ),
        final_layer_interval  # Perform interval arithmetic here
    )
    return model
end

function create_Q_Q_prime(affine_Q, task)
    # trainable parameters
    # println(affine_Q[1][1][2])
    # println(affine_Q[1][1][3])
    # println(affine_Q[2])
    
    # creating Q_prime
    affine_Q_interval = create_parallel_affine_Q_interval(task.x_dim, task.u_dim, task.u_low, task.u_high)
    println(affine_Q_interval)
    # Copy weights and biases from affine_Q to affine_Q_interval
    affine_Q_interval[1].layers[1].layers[2].weight .= affine_Q[1].layers[1].layers[2].weight
    affine_Q_interval[1].layers[1].layers[2].bias .= affine_Q[1].layers[1].layers[2].bias
    affine_Q_interval[1].layers[1].layers[3].weight .= affine_Q[1].layers[1].layers[3].weight
    affine_Q_interval[1].layers[1].layers[3].bias .= affine_Q[1].layers[1].layers[3].bias
    println("PASS 0")
    # Map the final layer to DenseInterval
    affine_Q_interval[2].W .= affine_Q[2].layers[1].weight
    affine_Q_interval[2].b .= affine_Q[2].layers[1].bias
    println("PASS 1")
    x_w = create_filter_matrix(1, task.x_dim, task.x_dim + task.u_dim)
    x_b = zeros(task.x_dim)
    filter_x = Dense(x_w, x_b)
    expand_layer = create_expand_xu_layer(task.x_dim, task.u_dim)
    println("PASS 2")
    return Chain(Parallel(+,
        Chain(affine_Q, Dense(Float32[1; 0;;])),
        Chain(filter_x, f_pi_model, expand_layer, affine_Q_interval, Dense(Float32[0; 1;;])),
    ))
end

Q_Q_prime = create_Q_Q_prime(affine_Q, task)
# println(Q_Q_prime)

input = Float32.(rand(task.x_dim + task.u_dim))
Q_Q_prime(input)



Chain(Parallel(vcat, Chain(Dense(5 => 3), Dense(3 => 32, relu), Dense(32 => 32, relu)), Dense(5 => 2)), DenseInterval([-2.5848196252384765 -2.2441538891180324 -0.029640842442632934 -1.0459967071975305 0.029323895642413743 0.2491336480656789 0.3311446206204225 0.508050848757809 0.6068861832273464 0.01934963500494938 1.7530919726006564 3.041078597052414 -0.2503978326050154 -0.9469995309488962 1.3602996690080662 0.5612062023558526 1.2793235524402076 1.517217063559812 -0.5110670601092122 0.24033374170978533 0.7029931531963389 -2.049401056841641 0.9025306363275714 -0.7315892175041379 0.31286263262289515 -0.8048340665169595 0.627643925955915 -0.7124762408080163 -0.01248051315316179 -0.6905358345729069 -1.4780301908074667 0.4304669095482064 -0.14959073715810042 0.8390414016976263], [-0.2864204163422486], 3, Float32[-1.0, -1.0], Float32[1.0, 1.0]))
PASS 0
PASS 1
PASS 2
32
(32,)


2-element Vector{Float32}:
 -0.19811821
 -0.30979547